<a href="https://colab.research.google.com/github/jingyuan0422/113-2-Programing-Language/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Authorize Google Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [52]:
# Output from GoogleSheet “reviews”
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1j7C_wFCTj3zWV0h_hjzfuApV9YH7zL0rHTEr2NNq0eU/edit?usp=sharing')

In [79]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1j7C_wFCTj3zWV0h_hjzfuApV9YH7zL0rHTEr2NNq0eU/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [56]:
from gspread_dataframe import set_with_dataframe

# Select the first worksheet
worksheet = gsheets.get_worksheet(0)

# return a 2D array
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])
df = df.iloc[:, [1, 2, 4]]  # 保留第2、3、5欄

# 去除 caption 欄位中的空行
df = df[df['caption'].str.strip() != '']

In [57]:
df

,caption,relative_date,rating
1,讚,10 小時前,5
2,好吃服務態度好,11 小時前,5
3,服務好 氣氛佳~,1 天前,5
14,(由 Google 提供翻譯) 完美的 (原始評論) Perfect,1 天前,5
18,這次的甜點不錯吃👍,2 天前,5
...,...,...,...
1091,服務親切，環境優雅，五星級實至名歸，,2 個月前,5
1092,服務與餐點都及格,2 個月前,5
1094,第一次來用餐加慶生 烤麵包蠻好吃的，不會太乾且可以續 然後這次來我點了 綠胡椒燻鴨胸溫沙拉、...,2 個月前,4
1095,體貼的服務，倍感溫馨！,2 個月前,5


In [58]:
df.describe()

,caption,relative_date,rating
count,644,644,644
unique,628,24,5
top,服務很好,1 個月前,5
freq,7,214,422


In [59]:
!pip install jieba
!pip install snownlp

In [80]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [81]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

In [82]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [83]:
df['Chinese_Content'] = df['caption'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [84]:
df['Chinese_Content']

,Chinese_Content
1,讚
2,好吃服務態度好
3,服務好氣氛佳
14,由提供翻譯完美的原始評論
18,這次的甜點不錯吃
...,...
1091,服務親切環境優雅五星級實至名歸
1092,服務與餐點都及格
1094,第一次來用餐加慶生烤麵包蠻好吃的不會太乾且可以續然後這次來我點了綠胡椒燻鴨胸溫沙拉松露蕈菇濃...
1095,體貼的服務倍感溫馨


In [85]:
df['Tokenized']

,Tokenized
1,[讚]
2,"[好吃, 服務, 態度, 好]"
3,"[服務, 好, 氣氛, 佳]"
14,"[由, 提供, 翻譯, 完美, 的, 原始, 評論]"
18,"[這次, 的, 甜點, 不錯, 吃]"
...,...
1091,"[服務, 親切, 環境, 優雅, 五星, 級, 實至名, 歸]"
1092,"[服務, 與, 餐點, 都, 及格]"
1094,"[第一次, 來, 用餐, 加慶生, 烤, 麵, 包蠻, 好吃, 的, 不會, 太乾且, 可以..."
1095,"[體貼, 的, 服務, 倍感, 溫馨]"


In [86]:
word_counts = Counter(all_words)
word_counts

Counter({'好吃': 188,
         '服務': 467,
         '態度': 33,
         '氣氛': 50,
         '提供': 59,
         '翻譯': 45,
         '完美': 6,
         '原始': 46,
         '評論': 47,
         '這次': 22,
         '甜點': 36,
         '不錯': 72,
         '服務品質': 34,
         '這裡': 3,
         '非常': 74,
         '上菜': 10,
         '速度': 6,
         '小朋友': 7,
         '照顧': 1,
         '更是': 7,
         '無微': 1,
         '大家': 4,
         '多多': 1,
         '支持': 2,
         '光顧': 3,
         '親切': 75,
         '餐點': 145,
         '豐富': 5,
         '選擇': 27,
         '食物': 28,
         '一定': 8,
         '水準': 20,
         '算是': 3,
         '王品': 124,
         '系列': 3,
         '艦店': 1,
         '一個': 20,
         '餐盤': 5,
         '不夠': 5,
         '乾淨': 13,
         '道歉': 2,
         '三人免': 1,
         '費海鮮': 1,
         '九折': 1,
         '誠意': 2,
         '不忍': 1,
         '苛責': 1,
         '佳環境': 1,
         '中旬': 1,
         '南京': 4,
         '東路店': 1,
         '用餐': 85,
         '親自體': 1,
         '驗

In [87]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set([
    '所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '非常', '真的', '評論', '原始', '翻譯',
    '感覺', '還是', '我們', '就是', '有點', '因為', '整體', '覺得', '這次', '部分', '但是', '一個', '這個', '沒有', '一樣', '今天', '還是', '我們', '就是',
    '一直', '只有', '雖然', '謝謝', '不是', '只是', '應該', '不要', '竟然', '超級', '這樣', '另外', '點了', '比較', '一點', '更是', '旁邊', '以往', '唯一',
    '一些', '是否', '除了', '什麼', '那麼', '當然', '依然', '依舊', '先生', '小姐', '貴店', '這家', '算是', '而言', '本身', '才能', '旁邊', '以往', '唯一',
    '提供', '以前', '蠻', '過', '有此'
                  ])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [88]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [89]:
word_freq_df

,Word,Frequency
0,服務,467
1,好吃,188
2,餐點,145
3,王品,124
4,牛排,91
...,...,...
2640,主餐點,1
2641,不韌,1
2642,熟軟,1
2643,獨特,1


In [75]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 1s (7,605 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [90]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [94]:
# 使用 Plotly Express 畫出 rating 的直方圖，並按照 5 到 1 排序
fig = px.histogram(df, x='rating', nbins=5, title='Distribution of Ratings')

# 設定 x 軸的排序
fig.update_xaxes(categoryorder='array', categoryarray=[5, 4, 3, 2, 1])

# 設定圖表樣式
fig.update_layout(
    xaxis_title='Rating',
    yaxis_title='Frequency',
    template='plotly',
    showlegend=False
)

# 在條形上加上頻次數字
fig.update_traces(texttemplate='%{y}', textposition='outside')

# 顯示圖表
fig.show()

In [95]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [96]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.0-flash")

prompt = f"請餐廳的顧客評論摘要出其重點，並分析其中的情緒表現。針對最多人提到的情緒或問題，提供具體的改善建議：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

## 餐廳顧客評論摘要與情緒分析

**摘要重點：**

*   **正面評價：**
    *   **服務優良:** 這是顧客評論中提及次數最多的優點，包含服務態度好、親切、貼心、周到、用心、有禮貌、快速、專業等。許多顧客對服務人員的個別表現表示讚賞 (例如：劉康鴻先生、孟婷等)。
    *   **餐點美味：** 許多顧客認為餐點好吃、美味、精緻、有水準，特別是牛排、甜點、湯品等。部分顧客提及特定餐點值得推薦 (例如：王品牛小排、犢牛肋排、法式牛尾清湯等)。
    *   **用餐環境佳：** 許多顧客稱讚餐廳的環境舒適、乾淨、整潔、優雅、有氣氛、燈光美、安靜等。
    *   **適合慶祝：** 許多顧客認為餐廳適合慶生、約會、家庭聚餐等特殊場合。
*   **負面評價：**
    *   **餐點品質不穩定：** 有部分顧客覺得餐點品質下降、不如以往，例如牛排過老、過柴、有筋、太油，龍蝦肉質過硬、不新鮮，湯品過鹹、過冷，海鮮清湯味道普通，甜點表現不佳等。
    *   **價格與價值不符：** 有部分顧客認為餐點價格偏高，但品質和份量未達期望，CP值不高。
    *   **服務細節不足：** 有部分顧客提到服務細節有待加強，例如上餐速度慢、對特殊需求 (如生日優惠、過敏) 的主動告知不足、桌面清理不夠、未及時續杯等。
    *   **環境問題：** 有部分顧客反映餐廳空調位置不佳、蚊蟲多、座位擁擠、隔音差、停車不便等。

**情緒表現分析：**

*   **正面情緒：** 滿意、開心、愉悅、感謝、讚賞、推薦。
*   **負面情緒：** 失望、不滿、生氣、氣憤、抱怨、不推薦。

**最多人提到的情緒或問題：**

1.  **服務品質下降：** 雖然多數顧客仍讚賞服務，但負面評論中，服務細節不足、服務人員不夠主動積極、處理問題不夠完善等情況被頻繁提及。
2.  **餐點品質不穩定：** 部分顧客反映餐點品質不一致，曾有餐點不新鮮、烹調不當、味道普通等情況發生，未能達到與價格相符的期待。
3.  **價格高昂但體驗不符：** 許多顧客認為王品的價格偏高，但餐點品質、服務細節、環境等方面未能提供相應的高價值體驗。

**具體改善建議：**

1.  **加強服務人員培訓：**
    *   **提升主動性和敏感度：** 訓練服務人員主動觀